In [25]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import gensim.downloader as api
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim import models 
from gensim.models import KeyedVectors, Word2Vec
import warnings
warnings.filterwarnings('ignore')


In [2]:
data_url = ("/Users/vidyakumar/Desktop/python/apziva/potential-talents - Aspiring human resources - seeking human resources.csv")
pd.set_option('display.max_colwidth', None)
get_ipython().run_line_magic('matplotlib', 'inline')

In [3]:
def load_data():
    data = pd.read_csv(data_url)
    return data

data = load_data()

data.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [4]:
# If it's the same job description in the same city, for the same job title, we consider it duplicate.
print(data.shape)
df_nodups = data.drop_duplicates(subset=['job_title', 'location'])
print(df_nodups.shape)

(104, 5)
(54, 5)


In [5]:
df_nodups.isnull().sum()

id             0
job_title      0
location       0
connection     0
fit           54
dtype: int64

In [6]:
df_nodups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 0 to 103
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          54 non-null     int64  
 1   job_title   54 non-null     object 
 2   location    54 non-null     object 
 3   connection  54 non-null     object 
 4   fit         0 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 2.5+ KB


In [7]:
#replace acronymns
df_nodups['job_title'] = df_nodups['job_title'].str.replace("HRIS", " Human Resources Information System ")
df_nodups['job_title'] = df_nodups['job_title'].str.replace("HR", " Human Resources ")
df_nodups['job_title'] = df_nodups['job_title'].str.replace("EPIK", " English Program in Korea ")
df_nodups['job_title'] = df_nodups['job_title'].str.replace("JTI", " Japan Tobacco International ")

#removing stopwords and making it all lowercase
stop_words = set(stopwords.words('english'))
df_nodups['job_title_no_stpw'] = df_nodups['job_title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df_nodups['job_title_no_stpw'] = df_nodups['job_title_no_stpw'].str.lower()
df_nodups.head(10)

,id,job_title,location,connection,fit,job_title_no_stpw
0,1,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,NaN,2019 c.t. bauer college business graduate (magna cum laude) aspiring human resources professional
1,2,Native English Teacher at English Program in Korea,Kanada,500+,NaN,native english teacher english program korea
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,aspiring human resources professional
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,people development coordinator ryan
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,advisory board member celal bayar university
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,aspiring human resources specialist
6,7,Student at Humber College and Aspiring Human Resources Generalist,Kanada,61,NaN,student humber college aspiring human resources generalist
7,8,Human Resources Senior Specialist,San Francisco Bay Area,500+,NaN,human resources senior specialist
9,10,Seeking Human Resources Human Resources Information System and Generalist Positions,Greater Philadelphia Area,500+,NaN,seeking human resources human resources information system generalist positions
10,11,Student at Chapman University,"Lake Forest, California",2,NaN,student chapman university


In [8]:
#removing digits and special characters
df_nodups['job_title_no_splc'] = df_nodups['job_title_no_stpw'].str.replace(r'[^A-Za-z0-9 ]+', '', regex=True)
df_nodups['job_title_no_splc'] = df_nodups['job_title_no_splc'].str.replace(r'\d+','') 
df_nodups.head()

,id,job_title,location,connection,fit,job_title_no_stpw,job_title_no_splc
0,1,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,NaN,2019 c.t. bauer college business graduate (magna cum laude) aspiring human resources professional,ct bauer college business graduate magna cum laude aspiring human resources professional
1,2,Native English Teacher at English Program in Korea,Kanada,500+,NaN,native english teacher english program korea,native english teacher english program korea
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,aspiring human resources professional,aspiring human resources professional
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,people development coordinator ryan,people development coordinator ryan
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,advisory board member celal bayar university,advisory board member celal bayar university


In [9]:
lemmer = nltk.stem.WordNetLemmatizer()

def lem(words):
    return " ".join([lemmer.lemmatize(word,'v') for word in words.split()])

df_nodups['job_lemmatized'] =  df_nodups.job_title_no_splc.apply(lem)

df_nodups.head()

,id,job_title,location,connection,fit,job_title_no_stpw,job_title_no_splc,job_lemmatized
0,1,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,NaN,2019 c.t. bauer college business graduate (magna cum laude) aspiring human resources professional,ct bauer college business graduate magna cum laude aspiring human resources professional,ct bauer college business graduate magna cum laude aspire human resources professional
1,2,Native English Teacher at English Program in Korea,Kanada,500+,NaN,native english teacher english program korea,native english teacher english program korea,native english teacher english program korea
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,aspiring human resources professional,aspiring human resources professional,aspire human resources professional
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,people development coordinator ryan,people development coordinator ryan,people development coordinator ryan
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,advisory board member celal bayar university,advisory board member celal bayar university,advisory board member celal bayar university


In [10]:
# Convert job_title column into a list
job_title_list = list(df_nodups['job_lemmatized'])                    

# Vectorize job_title_list
vectors = TfidfVectorizer()                                 
matrix = vectors.fit_transform(job_title_list)
feature_names = vectors.get_feature_names()  
print("Number of unique features: ", len(feature_names))   
print(feature_names)

Number of unique features:  175
['administration', 'administrative', 'admissions', 'advisory', 'always', 'america', 'an', 'analyst', 'analytics', 'army', 'arts', 'aspire', 'assistant', 'atlanta', 'bachelor', 'bauer', 'bayar', 'beach', 'beneteau', 'biology', 'board', 'brand', 'buckhead', 'business', 'care', 'celal', 'center', 'chapman', 'college', 'communications', 'community', 'compensation', 'conflict', 'coordinator', 'create', 'csr', 'ct', 'cum', 'customer', 'data', 'delphi', 'development', 'director', 'employment', 'endemol', 'energetic', 'energy', 'engage', 'engie', 'engineer', 'english', 'entrylevel', 'environment', 'environmental', 'excellence', 'executive', 'experience', 'ey', 'generalist', 'gi', 'gp', 'graduate', 'groupe', 'guard', 'hardware', 'heil', 'help', 'houston', 'human', 'humber', 'illinois', 'inc', 'inclusive', 'indiana', 'information', 'intelligence', 'intercontinental', 'international', 'internship', 'japan', 'junior', 'kokomo', 'korea', 'lab', 'laude', 'lead', 'lead

In [11]:
vectors.vocabulary_

{'ct': 36,
 'bauer': 15,
 'college': 28,
 'business': 23,
 'graduate': 61,
 'magna': 94,
 'cum': 37,
 'laude': 84,
 'aspire': 11,
 'human': 68,
 'resources': 134,
 'professional': 125,
 'native': 105,
 'english': 50,
 'teacher': 159,
 'program': 127,
 'korea': 82,
 'people': 120,
 'development': 41,
 'coordinator': 33,
 'ryan': 138,
 'advisory': 3,
 'board': 20,
 'member': 102,
 'celal': 25,
 'bayar': 16,
 'university': 166,
 'specialist': 149,
 'student': 153,
 'humber': 69,
 'generalist': 58,
 'senior': 143,
 'seek': 142,
 'information': 74,
 'system': 157,
 'position': 123,
 'chapman': 27,
 'svp': 156,
 'market': 99,
 'communications': 29,
 'csr': 35,
 'officer': 110,
 'engie': 48,
 'houston': 67,
 'the': 161,
 'woodlands': 172,
 'energy': 46,
 'gp': 60,
 'sp': 148,
 'intercontinental': 76,
 'buckhead': 22,
 'atlanta': 13,
 'management': 96,
 'internship': 78,
 'opportunities': 113,
 'experience': 56,
 'retail': 135,
 'manager': 98,
 'staff': 151,
 'recruit': 129,
 'luxottica': 93,


In [12]:
# Convert job titles into arrays
tfidf_vector = matrix.toarray()                                  
print("Shape of Tfidf vector: ", tfidf_vector.shape) 
print(tfidf_vector)

Shape of Tfidf vector:  (54, 175)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.46104593 0.         0.         ... 0.         0.         0.        ]]


In [13]:
print(tfidf_vector[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.19276841
 0.         0.         0.         0.3511579  0.         0.
 0.         0.         0.         0.         0.         0.26173515
 0.         0.         0.         0.         0.31815462 0.
 0.         0.         0.         0.         0.         0.
 0.3511579  0.3511579  0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.31815462 0.         0.         0.         0.
 0.         0.         0.11589292 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.3511579  0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.3511579  0.
 0.         0.         0.         0.    

In [14]:
# search query
search = "seeking human resources"


In [15]:
lemmer = nltk.stem.WordNetLemmatizer()
lem_search = " ".join([lemmer.lemmatize(word,'v') for word in search.split()])
print(lem_search)

seek human resources


In [16]:
# Convert search phrase into a vector
sv = vectors.transform([lem_search])                    
search_vector = sv.toarray()
print("Shape of search phrase vector:", search_vector.shape)
print(search_vector)

Shape of search phrase vector: (1, 175)
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.4319816  0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.


In [17]:
cosine_sim = cosine_similarity(tfidf_vector, search_vector)

df_nodups['tfidf_sim_score'] = cosine_sim
df_nodups.sort_values(by ='tfidf_sim_score', ascending = False).head()

,id,job_title,location,connection,fit,job_title_no_stpw,job_title_no_splc,job_lemmatized,tfidf_sim_score
98,99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,NaN,seeking human resources position,seeking human resources position,seek human resources position,0.696263
27,28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,NaN,seeking human resources opportunities,seeking human resources opportunities,seek human resources opportunities,0.673137
72,73,"Aspiring Human Resources Manager, seeking internship in Human Resources.","Houston, Texas Area",7,NaN,"aspiring human resources manager, seeking internship human resources.",aspiring human resources manager seeking internship human resources,aspire human resources manager seek internship human resources,0.624745
9,10,Seeking Human Resources Human Resources Information System and Generalist Positions,Greater Philadelphia Area,500+,NaN,seeking human resources human resources information system generalist positions,seeking human resources human resources information system generalist positions,seek human resources human resources information system generalist position,0.517021
26,27,Aspiring Human Resources Management student seeking an internship,"Houston, Texas Area",500+,NaN,aspiring human resources management student seeking internship,aspiring human resources management student seeking internship,aspire human resources management student seek internship,0.459454


In [18]:
#word2vec

In [22]:
# Using a pre-trained word2vec model
EMBEDDING_FILE = '/Users/vidyakumar/GoogleNews-vectors-negative300.bin'
word_vectors = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [ ]:
# Access vectors for specific words with a keyed lookup:
vectort = word_vectors['easy']
# see the shape of the vector (300,)
vectort.shape

In [23]:
# Tokenization of each document
tokenized_sent = []
for s in df_nodups['job_lemmatized']:
    tokenized_sent.append(word_tokenize(s.lower()))
tokenized_sent


[['ct',
  'bauer',
  'college',
  'business',
  'graduate',
  'magna',
  'cum',
  'laude',
  'aspire',
  'human',
  'resources',
  'professional'],
 ['native', 'english', 'teacher', 'english', 'program', 'korea'],
 ['aspire', 'human', 'resources', 'professional'],
 ['people', 'development', 'coordinator', 'ryan'],
 ['advisory', 'board', 'member', 'celal', 'bayar', 'university'],
 ['aspire', 'human', 'resources', 'specialist'],
 ['student',
  'humber',
  'college',
  'aspire',
  'human',
  'resources',
  'generalist'],
 ['human', 'resources', 'senior', 'specialist'],
 ['seek',
  'human',
  'resources',
  'human',
  'resources',
  'information',
  'system',
  'generalist',
  'position'],
 ['student', 'chapman', 'university'],
 ['svp',
  'c',
  'human',
  'resources',
  'o',
  'market',
  'communications',
  'csr',
  'officer',
  'engie',
  'houston',
  'the',
  'woodlands',
  'energy',
  'gp',
  'human',
  'resources',
  'sp',
  'human',
  'resources'],
 ['human',
  'resources',
  'coord

In [27]:
# train word2vec model
w2vec = Word2Vec(tokenized_sent, min_count=1, size=5)

print(w2vec)
#word2vec(vocab=19, size=5, alpha=0.025)


Word2Vec(vocab=177, size=5, alpha=0.025)


In [28]:
print(w2vec['humber'])

[ 0.04479725 -0.05589376  0.05632111  0.00475959  0.08110686]


In [29]:
w2vec.wv.vocab

{'ct': <gensim.models.keyedvectors.Vocab at 0x1049d47f0>,
 'bauer': <gensim.models.keyedvectors.Vocab at 0x1049d43d0>,
 'college': <gensim.models.keyedvectors.Vocab at 0x1049d4820>,
 'business': <gensim.models.keyedvectors.Vocab at 0x1049d41f0>,
 'graduate': <gensim.models.keyedvectors.Vocab at 0x1049d4340>,
 'magna': <gensim.models.keyedvectors.Vocab at 0x1049d4070>,
 'cum': <gensim.models.keyedvectors.Vocab at 0x1049d43a0>,
 'laude': <gensim.models.keyedvectors.Vocab at 0x1049d40d0>,
 'aspire': <gensim.models.keyedvectors.Vocab at 0x1049d4310>,
 'human': <gensim.models.keyedvectors.Vocab at 0x1049d4580>,
 'resources': <gensim.models.keyedvectors.Vocab at 0x1049d4160>,
 'professional': <gensim.models.keyedvectors.Vocab at 0x1049d4790>,
 'native': <gensim.models.keyedvectors.Vocab at 0x1049d49a0>,
 'english': <gensim.models.keyedvectors.Vocab at 0x1049d4a30>,
 'teacher': <gensim.models.keyedvectors.Vocab at 0x1049d4a60>,
 'program': <gensim.models.keyedvectors.Vocab at 0x1049d4ac0>,
 '